<a href="https://colab.research.google.com/github/semensorokin/data_augmetation/blob/master/LSTMandCNNEasyDataAug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

models from EDA https://github.com/jasonwei20/eda_nlp/tree/d75e8bd4631f4d93260cb291aa47852d8eacd51d/experiments
some parameters changed - size of model(lstm, CNN) layers 

In [1]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
import keras.layers as layers
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping
import keras.backend as K
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, f1_score

import math
import time
import numpy as np
import random
from random import randint
random.seed(3)
import datetime, re, operator
from random import shuffle
from time import gmtime, strftime
import gc
import keras
import tensorflow as tf

import os
from os import listdir
from os.path import isfile, join, isdir
import pickle

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12940136102339084745
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12108141553435784198
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6549464723341371551
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7304675328
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13292425830433952609
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"
]


In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/UniversalEmb

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/UniversalEmb


In [0]:
!rm wiki-news-300d-1M-subword.vec.zip

In [10]:
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M-subword.vec', binary=False) 

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import pandas as pd
def read_trec(n_examples, typ = 1):
  "type = 0 - trec6, type = 1 - trec50"
  labels = [i.split(' ', 1 )[0].split(':')[typ] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
  texts = [ i.split(' ', 1 )[1][:-1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
  train  = pd.DataFrame({'text':texts, 'labels': labels})
  lbl2indx = {i:j for j, i in enumerate(train.labels.unique())}
  train['lbl_index'] = train.labels.apply(lambda x:lbl2indx[x])

  labels_t = [i.split(' ', 1)[0].split(':')[typ] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  test  = pd.DataFrame({'text':texts_t, 'labels': labels_t})
  test['lbl_index'] = test.labels.apply(lambda x:lbl2indx[x])

  return train, test, lbl2indx

In [0]:
def transform(data, w2i, n_classes, input_size):
  x_matrix = np.zeros((data.shape[0], input_size))
  y_matrix = np.zeros((data.shape[0], n_classes))
  sent_len = []
  found_word, unk_w = 0,0
  for i, line in enumerate(data.iterrows()):
    label = line[1]['lbl_index']
    sentence = line[1]['text']
    splited_sent = sentence.split(' ')
    sent_len.append(len(splited_sent))

    for enum, word in enumerate(splited_sent[:input_size]):
      if word in w2i:
        x_matrix[i][enum] = w2i[word]
        found_word+=1
      else:
        x_matrix[i][enum] = w2i['unk']
        unk_w+=1
    y_matrix[i][label] = 1.0
  print('Average sentence len: ', np.mean(sent_len))
  print('Max sentence len: ', np.max(sent_len) )
  print('{0:0.3f}% of tokens were replaced by unk token index'.format((unk_w/(unk_w+found_word))*100))

  return x_matrix.astype(int), y_matrix 


def create_train_index(tr, w2v_model, max_s_l=128):
  w2i = {'pad': 0, 'unk': 1}

  #create tokens list and types
  tokens = []
  for indx, line in tr.iterrows():
    tokens.extend(line['text'].split(' '))
  types = list(set(tokens))
  #create w2i vocab and pretrained emb matrix
  emb_matrix = []
  emb_matrix.append(np.zeros(300)) #'pad'
  emb_matrix.append(np.zeros(300)) #'unk'
  s,d = 0,0
  for i, t in enumerate(types):
    if t in w2v_model:
      indx = len(w2i)
      w2i[t] = indx
      emb_matrix.append(w2v_model[t])
      d += 1
    else:
      s += 1
  print('Not found vectors {} out of {} unique words'.format(s, d+s))
  i2w = {j:i for i, j in w2i.items()}
  #transform train tokens to index in each text
  x_tr, y_tr = transform(tr, w2i, len(tr.lbl_index.unique()), max_s_l)
  return x_tr, y_tr, w2i, i2w, np.array(emb_matrix)

In [0]:
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [0]:
#building the model in keras with embeddings
def build_lstm(word2vec_matrix, max_seq_len, num_classes):
  model = None
  model = Sequential()
  model.add(layers.Embedding(word2vec_matrix.shape[0], word2vec_matrix.shape[1], input_length=max_seq_len,  weights=[word2vec_matrix], trainable=False))
  model.add(Bidirectional(LSTM(256, return_sequences=True)))
  model.add(Dropout(0.5))
  model.add(Bidirectional(LSTM(128, return_sequences=False)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1 ])
  #print(model.summary())
  return model

In [0]:
def one_hot_to_categorical(y):
    assert len(y.shape) == 2
    return np.argmax(y, axis=1)

In [0]:
def run_lstm(train_data, test_data, w2v_model, msl, n_c):

  #load data
  train_x, train_y, word2in, in2word, pretrained_vectors = create_train_index(train_data, w2v_model, msl)
  test_x, test_y = transform(test_data, word2in, len(train_data.lbl_index.unique()), train_x.shape[1])

  #initialize model
  model = build_lstm(pretrained_vectors, msl, n_c)

  #implement early stopping
  callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

  #train model
  model.fit(train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')

  #evaluate model
  y_pred = model.predict(test_x)
  test_y_cat = one_hot_to_categorical(test_y)
  y_pred_cat = one_hot_to_categorical(y_pred)
  f1 = f1_score(test_y_cat, y_pred_cat, average='micro')
  acc = accuracy_score(test_y_cat, y_pred_cat)

  print('Best F1:{}, best accuracy:{}'.format(acc, f1))
  return acc

In [0]:
#building the cnn in keras
def build_cnn(word2vec_matrix, max_seq_len, num_classes):
  model = None
  model = Sequential()
  model.add(layers.Embedding(word2vec_matrix.shape[0], word2vec_matrix.shape[1], input_length=max_seq_len,  weights=[word2vec_matrix]))
  model.add(layers.Conv1D(256, 5, activation='relu'))
  model.add(layers.GlobalMaxPooling1D())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1])
  return model

In [0]:
def run_cnn(train_data, test_data, w2v_model, msl, n_c):

  #load data
  train_x, train_y, word2in, in2word, pretrained_vectors = create_train_index(train_data, w2v_model, msl)
  test_x, test_y = transform(test_data, word2in, len(train_data.lbl_index.unique()), train_x.shape[1])

  #initialize model
  model = build_cnn(pretrained_vectors, msl, n_c)

  #implement early stopping
  callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

  #train model
  model.fit(train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')

  #evaluate model
  y_pred = model.predict(test_x)
  test_y_cat = one_hot_to_categorical(test_y)
  y_pred_cat = one_hot_to_categorical(y_pred)
  f1 = f1_score(test_y_cat, y_pred_cat, average='micro')
  acc = accuracy_score(test_y_cat, y_pred_cat)

  print('Best F1:{}, best accuracy:{}'.format(acc, f1))
  return acc

#Trec-50 

In [0]:
train_data, test_data, l2i = read_trec(6000, 1)

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [144]:
run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )

Not found vectors 262 out of 9448 unique words
Average sentence len:  10.204512105649304
Max sentence len:  37
2.017% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.792% of tokens were replaced by unk token index
Train on 5452 samples, validate on 500 samples
Epoch 1/100000
5452/5452 [==============================] - 5s 840us/step - loss: 3.6479 - accuracy: 0.1282 - f1: 0.0000e+00 - val_loss: 3.2121 - val_accuracy: 0.1100 - val_f1: 0.0000e+00
Epoch 2/100000
5452/5452 [==============================] - 3s 613us/step - loss: 3.1220 - accuracy: 0.1653 - f1: 0.0000e+00 - val_loss: 3.2496 - val_accuracy: 0.1100 - val_f1: 0.0000e+00
Epoch 3/100000
5452/5452 [==============================] - 3s 598us/step - loss: 3.0808 - accuracy: 0.1669 - f1: 0.0000e+00 - val_loss: 3.2170 - val_accuracy: 0.1120 - val_f1: 0.0000e+00
Epoch 4/100000
5452/5452 [==============================] - 3s 584us/step - loss: 3.0653 - accuracy: 0.1708 - f1: 0.0000e+00 - 

0.602

In [147]:
run_cnn(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES)

Not found vectors 262 out of 9448 unique words
Average sentence len:  10.204512105649304
Max sentence len:  37
2.017% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.792% of tokens were replaced by unk token index


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 5452 samples, validate on 500 samples
Epoch 1/100000
5452/5452 [==============================] - 0s 84us/step - loss: 3.7233 - accuracy: 0.1434 - f1: 0.0000e+00 - val_loss: 3.4701 - val_accuracy: 0.1100 - val_f1: 0.0000e+00
Epoch 2/100000
5452/5452 [==============================] - 0s 49us/step - loss: 3.1823 - accuracy: 0.1772 - f1: 0.0000e+00 - val_loss: 3.1358 - val_accuracy: 0.2460 - val_f1: 0.0000e+00
Epoch 3/100000
5452/5452 [==============================] - 0s 49us/step - loss: 2.9005 - accuracy: 0.2937 - f1: 0.0000e+00 - val_loss: 2.9211 - val_accuracy: 0.4420 - val_f1: 0.0000e+00
Epoch 4/100000
5452/5452 [==============================] - 0s 49us/step - loss: 2.5817 - accuracy: 0.3298 - f1: 0.0777 - val_loss: 2.5325 - val_accuracy: 0.4440 - val_f1: 0.2292
Epoch 5/100000
5452/5452 [==============================] - 0s 48us/step - loss: 2.1574 - accuracy: 0.4461 - f1: 0.3568 - val_loss: 2.1968 - val_accuracy: 0.4900 - val_f1: 0.4882
Epoch 6/100000
5452/5452 [========

0.748

### TREC-50 with augmetaion

In [0]:
train_data, test_data, l2i = read_trec(6000, 1)

In [0]:
def read_aug(name, lbl2indx):
  aug = pd.read_csv(name, index_col=0)
  aug['lbl_index'] = aug.labels.apply(lambda x:lbl2indx[x])
  aug = aug[['texts', 'lbl_index']]
  aug.columns = ['text', 'lbl_index']
  return aug

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [0]:
augmentation = read_aug('trec50_n1_bert_squad_pr_b-0.5_min_b-min_max_b-max_coefs2-2.csv', l2i)

In [215]:
augmentation

,text,lbl_index
0,"After her second solo album, what other entert...",14
1,Beyonce's younger sibling also sang with her i...,5
2,What town did Beyonce go to school in?,21
3,Who decided to place Beyonce's group in Star S...,4
4,"""Charlie's Angels"" featured which single from ...",1
...,...,...
89147,Large quantities of what can be created for te...,14
89148,What did Parmenides say everything was made of?,28
89149,How does Descartes use matter and the formal/f...,0
89150,Both primary and secondary properties are suit...,14


In [216]:
run_lstm(pd.concat([train_data, augmentation]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES )

Not found vectors 27947 out of 65944 unique words
Average sentence len:  10.32127605598072
Max sentence len:  25601
12.269% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
4.604% of tokens were replaced by unk token index
Train on 94604 samples, validate on 500 samples
Epoch 1/100000
94604/94604 [==============================] - 56s 590us/step - loss: 2.4234 - accuracy: 0.3710 - f1: 0.1354 - val_loss: 3.1244 - val_accuracy: 0.2700 - val_f1: 0.1812
Epoch 2/100000
94604/94604 [==============================] - 55s 584us/step - loss: 1.8258 - accuracy: 0.5342 - f1: 0.4015 - val_loss: 2.5717 - val_accuracy: 0.3500 - val_f1: 0.2968
Epoch 3/100000
94604/94604 [==============================] - 55s 583us/step - loss: 1.6626 - accuracy: 0.5683 - f1: 0.4763 - val_loss: 2.3491 - val_accuracy: 0.3640 - val_f1: 0.3571
Epoch 4/100000
94604/94604 [==============================] - 55s 582us/step - loss: 1.5664 - accuracy: 0.5811 - f1: 0.5244 - val_loss:

0.824

In [219]:
run_cnn(pd.concat([train_data, augmentation]), test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES)

Not found vectors 27947 out of 65944 unique words
Average sentence len:  10.32127605598072
Max sentence len:  25601
12.269% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
4.604% of tokens were replaced by unk token index


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 94604 samples, validate on 500 samples
Epoch 1/100000
94604/94604 [==============================] - 6s 66us/step - loss: 1.5710 - accuracy: 0.6126 - f1: 0.5381 - val_loss: 1.3110 - val_accuracy: 0.7560 - val_f1: 0.7776
Epoch 2/100000
94604/94604 [==============================] - 6s 64us/step - loss: 0.5106 - accuracy: 0.8664 - f1: 0.8684 - val_loss: 0.9257 - val_accuracy: 0.8100 - val_f1: 0.8364
Epoch 3/100000
94604/94604 [==============================] - 6s 64us/step - loss: 0.2475 - accuracy: 0.9341 - f1: 0.9357 - val_loss: 0.8187 - val_accuracy: 0.8300 - val_f1: 0.8443
Epoch 4/100000
94604/94604 [==============================] - 6s 64us/step - loss: 0.1243 - accuracy: 0.9676 - f1: 0.9683 - val_loss: 0.8806 - val_accuracy: 0.8380 - val_f1: 0.8453
Epoch 5/100000
94604/94604 [==============================] - 6s 64us/step - loss: 0.0679 - accuracy: 0.9836 - f1: 0.9839 - val_loss: 0.9141 - val_accuracy: 0.8540 - val_f1: 0.8598
Epoch 6/100000
94604/94604 [===================

0.848

#Trec-6

In [0]:
train_data, test_data, l2i = read_trec(6000, typ=0)

In [0]:
MAX_SEQ_LEN, N_CLASSES = 37, len(train_data.lbl_index.unique())

In [150]:
run_lstm(train_data, test_data, w2v_model, MAX_SEQ_LEN, N_CLASSES)

Not found vectors 262 out of 9448 unique words
Average sentence len:  10.204512105649304
Max sentence len:  37
2.017% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.792% of tokens were replaced by unk token index
Train on 5452 samples, validate on 500 samples
Epoch 1/100000
5452/5452 [==============================] - 4s 807us/step - loss: 1.7272 - accuracy: 0.2273 - f1: 0.0000e+00 - val_loss: 1.6620 - val_accuracy: 0.2960 - val_f1: 0.0000e+00
Epoch 2/100000
5120/5452 [===========================>..] - ETA: 0s - loss: 1.6487 - accuracy: 0.2510 - f1: 0.0000e+00

KeyboardInterrupt: ignored

In [152]:
run_cnn(train_data, test_data, w2v_model,MAX_SEQ_LEN, N_CLASSES)

Not found vectors 262 out of 9448 unique words
Average sentence len:  10.204512105649304
Max sentence len:  37
2.017% of tokens were replaced by unk token index
Average sentence len:  7.516
Max sentence len:  17
9.792% of tokens were replaced by unk token index


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 5452 samples, validate on 500 samples
Epoch 1/100000
5452/5452 [==============================] - 0s 86us/step - loss: 1.7058 - accuracy: 0.2735 - f1: 0.0000e+00 - val_loss: 1.6018 - val_accuracy: 0.4400 - val_f1: 0.0000e+00
Epoch 2/100000
5452/5452 [==============================] - 0s 49us/step - loss: 1.5215 - accuracy: 0.4182 - f1: 0.0080 - val_loss: 1.3221 - val_accuracy: 0.4780 - val_f1: 0.3121
Epoch 3/100000
5452/5452 [==============================] - 0s 48us/step - loss: 1.2099 - accuracy: 0.6016 - f1: 0.2855 - val_loss: 0.9695 - val_accuracy: 0.6500 - val_f1: 0.5501
Epoch 4/100000
5452/5452 [==============================] - 0s 47us/step - loss: 0.7607 - accuracy: 0.7727 - f1: 0.6672 - val_loss: 0.6365 - val_accuracy: 0.7940 - val_f1: 0.7523
Epoch 5/100000
5452/5452 [==============================] - 0s 47us/step - loss: 0.3716 - accuracy: 0.9116 - f1: 0.8998 - val_loss: 0.3953 - val_accuracy: 0.8620 - val_f1: 0.8640
Epoch 6/100000
5452/5452 [========================

0.9

#SST-2

In [0]:
# SST2 https://www.kaggle.com/atulanandjha/distillbert-extensive-tutorial-starter-kernel/data

In [0]:
import pandas as pd
def read_sst2(n_examples, use_dev=False):
  train = pd.read_csv('SST/train.tsv', sep='\t')[:n_examples]
  if use_dev:
    dev = pd.read_csv('SST/dev.tsv', sep='\t')
    train = pd.concat([train, dev]).reset_index(drop=True)[:n_examples]
    print(train.head())
    train.columns = ['text', 'lbl_index']
  train.columns = ['text', 'lbl_index']
  test = pd.read_csv('SST/test.tsv', sep='\t')
  test.columns = ['text', 'lbl_index']
  return train, test, {'neg': 0, 'pos': 1}

In [154]:
train_sst, test_sst, lbl2ind_sst = read_sst2(10000, True)

                                             Reviews  Ratings
0  a stirring , funny and finally transporting re...        1
1  apparently reassembled from the cutting room f...        0
2  they presume their audience wo n't sit still f...        0
3  this is a visually stunning rumination on love...        1
4  jonathan parker 's bartleby should have been t...        1


In [156]:
run_cnn(train_sst, test_sst, w2v_model, 53, 2)

Not found vectors 809 out of 14648 unique words
Average sentence len:  18.535292607802873
Max sentence len:  51
2.344% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
6.677% of tokens were replaced by unk token index


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7792 samples, validate on 1821 samples
Epoch 1/100000
7792/7792 [==============================] - 1s 140us/step - loss: 0.6841 - accuracy: 0.5675 - f1: 0.5703 - val_loss: 0.6638 - val_accuracy: 0.6107 - val_f1: 0.6099
Epoch 2/100000
7792/7792 [==============================] - 1s 66us/step - loss: 0.5678 - accuracy: 0.7465 - f1: 0.7489 - val_loss: 0.4691 - val_accuracy: 0.7787 - val_f1: 0.7772
Epoch 3/100000
7792/7792 [==============================] - 1s 66us/step - loss: 0.2824 - accuracy: 0.8928 - f1: 0.8941 - val_loss: 0.4311 - val_accuracy: 0.8193 - val_f1: 0.8215
Epoch 4/100000
7792/7792 [==============================] - 1s 65us/step - loss: 0.1160 - accuracy: 0.9607 - f1: 0.9613 - val_loss: 0.4924 - val_accuracy: 0.8155 - val_f1: 0.8165
Epoch 5/100000
7792/7792 [==============================] - 1s 65us/step - loss: 0.0434 - accuracy: 0.9906 - f1: 0.9911 - val_loss: 0.5902 - val_accuracy: 0.8045 - val_f1: 0.8052
Epoch 6/100000
7792/7792 [==============================

0.7990115321252059

In [157]:
run_lstm(train_sst, test_sst, w2v_model, 53, 2)

Not found vectors 809 out of 14648 unique words
Average sentence len:  18.535292607802873
Max sentence len:  51
2.344% of tokens were replaced by unk token index
Average sentence len:  18.480505216913784
Max sentence len:  53
6.677% of tokens were replaced by unk token index
Train on 7792 samples, validate on 1821 samples
Epoch 1/100000
7792/7792 [==============================] - 8s 988us/step - loss: 0.6882 - accuracy: 0.5424 - f1: 0.5412 - val_loss: 0.6760 - val_accuracy: 0.5854 - val_f1: 0.5890
Epoch 2/100000
7792/7792 [==============================] - 6s 808us/step - loss: 0.6585 - accuracy: 0.6127 - f1: 0.6155 - val_loss: 0.6063 - val_accuracy: 0.6798 - val_f1: 0.6803
Epoch 3/100000
7792/7792 [==============================] - 6s 804us/step - loss: 0.5896 - accuracy: 0.6902 - f1: 0.6905 - val_loss: 0.5996 - val_accuracy: 0.6749 - val_f1: 0.6752
Epoch 4/100000
7792/7792 [==============================] - 6s 812us/step - loss: 0.5473 - accuracy: 0.7284 - f1: 0.7286 - val_loss: 0.5

0.7825370675453048

OLD EDA code without batching embeding vectors

In [0]:
#getting the x and y inputs in numpy array form from the text file
def get_x_y(train_data, num_classes, word2vec, word2vec_len=300, input_size=64):
  num_lines = train_data.shape[0]

  #initialize x and y matrix
  x_matrix = None
  y_matrix = None

  try:
    x_matrix = np.zeros((num_lines, input_size, word2vec_len))
  except:
    print("Error!", num_lines, input_size, word2vec_len)
  y_matrix = np.zeros((num_lines, num_classes))
  s,d = 0,0
  #insert values
  for i, line in enumerate(train_data.iterrows()):
    label = line[1]['lbl_index']
    sentence = line[1]['texts']

    #insert x
    words = sentence.split(' ')
    words = words[:x_matrix.shape[1]] #cut off if too long
    for j, word in enumerate(words):
      if word in word2vec:
        x_matrix[i, j, :] = word2vec[word]
        d += 1
      else:
        s += 1

    #insert y
    y_matrix[i][label] = 1.0
  print('Not found vectors {} out of {} words in dateset'.format(s, d))
  return x_matrix, y_matrix

In [0]:
###################################################
##################### model #######################
###################################################

#building the model in keras
def build_model(sentence_length, word2vec_len, num_classes):
	model = None
	model = Sequential()
 
	model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(sentence_length, word2vec_len)))
	model.add(Dropout(0.5))
	model.add(Bidirectional(LSTM(128, return_sequences=False)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1 ])
	#print(model.summary())
	return model

#building the cnn in keras
def build_cnn(sentence_length, word2vec_matrix, max_seq_len, num_classes):
	model = None
	model = Sequential()
  model.add(Embedding(word2vec_matrix.shape[0], word2vec_matrix[1], input_length=max_seq_len,  weights=[word2vec_matrix]))
	model.add(layers.Conv1D(256, 5, activation='relu', input_shape=(sentence_length, word2vec_len)))
	model.add(layers.GlobalMaxPooling1D())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1])
	return model

In [0]:
def run_model(train_data, test_data, num_classes, word2vec, input_size=64, word2vec_len=300):

  #initialize model
  model = build_model(input_size, word2vec_len, num_classes)

  #load data
  train_x, train_y = get_x_y(train_data, num_classes, w2v_model)
  test_x, test_y = get_x_y(test_data, num_classes, w2v_model)

  #implement early stopping
  callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

  #train model
  model.fit(train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=512, 
        shuffle=True, 
        verbose=1)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')

  #evaluate model
  y_pred = model.predict(test_x)
  test_y_cat = one_hot_to_categorical(test_y)
  y_pred_cat = one_hot_to_categorical(y_pred)
  f1 = f1_score(test_y_cat, y_pred_cat, average='micro')
  acc = accuracy_score(test_y_cat, y_pred_cat)

  print('Best F1:{}, best accuracy:{}'.format(acc, f1))
  return acc

In [0]:
def run_cnn(train_data, test_data, num_classes, word2vec, input_size =64, word2vec_len=300):

  #initialize model
  model = build_cnn(input_size, word2vec_len, num_classes)

  #load data
  train_x, train_y = get_x_y(train_data, num_classes, w2v_model)
  test_x, test_y = get_x_y(test_data, num_classes, w2v_model)

  #implement early stopping
  callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

  #train model
  model.fit(	train_x, 
        train_y, 
        epochs=100000, 
        callbacks=callbacks,
        validation_data=(test_x, test_y), 
        batch_size=1024, 
        shuffle=True, 
        verbose=1)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')

  #evaluate model
  
  y_pred = model.predict(test_x)
  test_y_cat = one_hot_to_categorical(test_y)
  y_pred_cat = one_hot_to_categorical(y_pred)
  f1 = f1_score(test_y_cat, y_pred_cat, average='micro')
  acc = accuracy_score(test_y_cat, y_pred_cat)
  
  print('Best F1:{}, best accuracy:{}'.format(acc, f1))

  #clean memory???
  train_x, train_y, model = None, None, None
  gc.collect()

  #return the accuracy
  #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
  return acc